In [1]:
import keras
from keras import backend as K
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras.applications import mobilenet

import matplotlib.pyplot as plt
%matplotlib inline

/home/prudhvi/.conda/envs/tensorflow/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/prudhvi/.conda/envs/tensorflow/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/prudhvi/.conda/envs/tensorflow/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
K.set_learning_phase(0)  # all new operations will be in test mode from now on

In [3]:
def embedding_loss(y_true, y_pred):    
    
    return K.mean(K.sum(K.square(y_true - y_pred), axis = 1))

In [4]:
with CustomObjectScope({'relu6': mobilenet.relu6, 'embedding_loss' : embedding_loss}):
    model = load_model('/var/tmp/mobnet_distill.h5')

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 162, 162, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 80, 80, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 82, 82, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
__________

In [10]:
model.input.op.name

'input_1'

In [11]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

sess = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), [model.output.op.name])
graph_io.write_graph(constant_graph, './', 'train.pb', as_text=False)
print('saved the freezed graph (ready for inference)')

INFO:tensorflow:Froze 137 variables.
INFO:tensorflow:Converted 137 variables to const ops.
saved the freezed graph (ready for inference)
